In [2]:
import pandas as pd
import numpy as np

In [24]:
Country = 'Netherlands'

data = pd.read_csv(r'C:\Users\JMSch\Documents\MEGA\05_Paris\ESCP Business School\MSc Big Data & Business Analytics\COVID19\COVID19\Data\Extracts\Key_Logistic_Prediction_Output_Global_20200402_21h23.csv', sep = ',')
data = data[1:]
data = data.filter(['Country', 'Confirmed', 'Predictions'])
data = data.dropna()

In [23]:
ŷ = data['Predictions'].values
y = data['Confirmed'].values

correlation_matrix = np.corrcoef(ŷ, y)
correlation_xy = correlation_matrix[0,1]
r_squared = correlation_xy**2

print(f'The R-2 for {Country} = {r_squared}')

The R-2 for Netherlands = 0.99825035924343


In [25]:
Countries = data['Country'].tolist()
Countries = list(dict.fromkeys(Countries))

In [35]:
Full_Data = pd.DataFrame()

## For each country, find the 'Time' reference since patient 1, and the corrsponding date
for Country in Countries:
    data_cntry = data[data.Country.eq(Country)]


    ŷ = data_cntry['Predictions'].values
    y = data_cntry['Confirmed'].values

    n = data_cntry.count(axis='columns').count()
    p = 1

    correlation_matrix = np.corrcoef(ŷ, y)
    correlation_xy = correlation_matrix[0,1]
    r_squared = correlation_xy**2

    adj_r_squared = 1-(1-r_squared)*(n-1)/(n-p-1)


    print(f'The R^2 for {Country} =')
    print("%.4f" % r_squared)
    print("%.4f" % adj_r_squared)


The R^2 for Afghanistan =
0.9745
0.9729
The R^2 for Albania =
0.9905
0.9900
The R^2 for Algeria =
0.9946
0.9944
The R^2 for Andorra =
0.2885
0.2376
The R^2 for Argentina =
0.9921
0.9918
The R^2 for Armenia =
0.3519
0.3138
The R^2 for Austria =
0.9965
0.9964
The R^2 for Azerbaijan =
0.9961
0.9959
The R^2 for Bahamas =
0.9388
0.9082
The R^2 for Bahrain =
0.9896
0.9893
The R^2 for Bangladesh =
0.2044
0.1432
The R^2 for Barbados =
0.3075
0.2305
The R^2 for Belarus =
0.9256
0.9217
The R^2 for Belgium =
0.9962
0.9960
The R^2 for Benin =
nan
nan
The R^2 for Bolivia =
0.4177
0.3789
The R^2 for Bosnia and Herzegovina =
0.9930
0.9926
The R^2 for Brazil =
0.9893
0.9888
The R^2 for Brunei =
0.9254
0.9216
The R^2 for Bulgaria =
0.9813
0.9803
The R^2 for Burkina Faso =
0.3178
0.2724
The R^2 for Burma =
0.4518
-0.0965
The R^2 for Cambodia =
0.1341
0.0722
The R^2 for Cameroon =
0.6020
0.5688
The R^2 for Canada =
0.9979
0.9979
The R^2 for Chile =
0.8682
0.8620
The R^2 for China =
0.9854
0.9852
The R^2 

In [33]:
test = Full_Data[Full_Data.Country.eq(Country)]

In [34]:
    # function to be minimized
def f_sigmoid(x, a, b, c):
    # a = sigmoid midpoint
    # b = curve steepness (logistic growth)
    # c = max value
        return (c / (1 + np.exp(-b*(x-a))))

In [51]:
## building the model per country
Pred_Data = pd.DataFrame()
#Full_Data = pd.DataFrame()

for Country in Countries:
    data_cntry = Full_Data[Full_Data.Country.eq(Country)]
    if data_cntry.count(axis='columns').count() > 0:
        try:
            
            data_cntry = data_cntry[['Country', 'Time','Confirmed','Date']]
  
            Model_Data = data_cntry.dropna()
            x = Model_Data['Time'].values
            y = Model_Data['Confirmed'].values

            print(Country)
            # fitting the data on the logistic function
    	    # A: sigmoid midmoint
            # B: curve steepness (logistic growth)
            # C: max value
            popt_sig, pcov_sig = curve_fit(f_sigmoid, x, y, method='dogbox', bounds=([20., 0.001, y.mean()],[60., 2.5, 10*y.max()]), maxfev=5000)
            #print(popt_sig)

            Pred_Length = data_cntry['Confirmed'].values

            x_m = np.arange(len(Pred_Length))
            y_m = f_sigmoid(x_m, *popt_sig)

            data_cntry['Predictions']=pd.Series(y_m)

            Pred_Data = pd.concat([Pred_Data,data_cntry]) 

        except:
           continue

    else:
            continue


France


In [52]:
Pred_Data[50:70]

,Country,Time,Confirmed,Date,Predictions
50,France,51,40708.0,29/03/2020,37660.044304
51,France,52,45170.0,30/03/2020,42231.281592
52,France,53,52827.0,31/03/2020,47003.564654
53,France,54,57749.0,01/04/2020,51910.241357
54,France,55,NaN,02/04/2020,56876.533068
55,France,56,NaN,03/04/2020,61823.948474
56,France,57,NaN,04/04/2020,66675.179003
57,France,58,NaN,05/04/2020,71358.839581
58,France,59,NaN,06/04/2020,75813.452367
59,France,60,NaN,07/04/2020,79990.232358


In [44]:
now = datetime.now() # current date and time

Now = now.strftime("%Y%m%d_%Hh%M")

Pred_Data.to_csv(f'C:\\Users\\JMSch\\Documents\\MEGA\\05_Paris\\ESCP Business School\\MSc Big Data & Business Analytics\\COVID19\\COVID19\\Data\\Extracts\\Logistic_Prediction_Output_Global_{Country}.csv', index = False)